Importo tutte le librerie necessarie

In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 


from sklearn.metrics import confusion_matrix, accuracy_score
 

from keras.utils import to_categorical #to create dummy variable
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
import time

import PIL
import os
import cv2
import itertools
import glob

from sklearn.svm import SVC

import warnings 
warnings.filterwarnings('ignore')

Definisco la funzione per prelevare le immagini e creare il dataset


In [24]:
def read_image_from(path, label):
    y = [] #type of image
    x = [] #image

    for idx, i in enumerate(label):
        for im in glob.glob(path + '/' + i + '/*.jpg', recursive=True):
            # color conversion to black and white images
            x.append(cv2.cvtColor(cv2.imread(im), cv2.COLOR_BGR2GRAY))
            y.append(idx)

    return x, y

Inizio a creare le variabili di lavoro

In [25]:
#path to the directories [Adriano]
PATH_TRAIN=r'C:\Users\adria\Desktop\CVPR\Imageset\train'
PATH_TEST=r'C:\Users\adria\Desktop\CVPR\Imageset\test'

#path to the directories [Giovanni]
#PATH_TRAIN=r'C:\Users\adria\Desktop\CVPR\Imageset\train'
#PATH_TEST=r'C:\Users\adria\Desktop\CVPR\Imageset\tes'

#array of the class
labels=[os.path.basename(i) for i in glob.glob(PATH_TRAIN + '/*', recursive=True)]
num_classes = len(LABELS) #15
print("Classes number are: ", num_classes)
print("Class names: ", labels)

Classes number are:  15
Class names:  ['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']


In [26]:
X_train_raw, y_train_raw = read_image_from(PATH_TRAIN, labels) #get train dataset
X_test, y_test = read_image_from(PATH_TEST, labels) #get test dataset 
print('Image Dimensions : ',X_test[0].shape)
print('len train x: ',len(X_train_raw))
print('len test x: ',len(X_test))
print('len train y: ',len(y_train_raw))
print('len test y: ',len(y_test))

Image Dimensions :  (200, 276)
len train x:  1500
len test x:  2985
len train y:  1500
len test y:  2985
